In [15]:
import requests
import base64
import json
import os
from datetime import datetime, timedelta
from IPython.display import display, HTML
import pandas as pd

In [26]:
CLIENT_ID = "ABb0MBnMfSJKhNwwMY2zel8cL61bMGtuPmotbPxb9pv5Zn50yn"  # From Intuit Developer Dashboard
CLIENT_SECRET = "GjzZI2O9kL5Rfi6th3hcvRBskpb814Kfaz90VuEi"  # From Intuit Developer Dashboard
REDIRECT_URI = "https://developer.intuit.com/v2/OAuth2Playground/RedirectUrl"  # For testing
COMPANY_ID = "9341454675225450"  # Your QuickBooks company ID
IS_SANDBOX = True  # Set to False for production

REFRESH_TOKEN = "RT1-136-H0-1755970682gujh0lb5nvlfugovcotz"

if IS_SANDBOX:
    BASE_URL = "https://sandbox-quickbooks.api.intuit.com"
else:
    BASE_URL = "https://quickbooks.api.intuit.com"

AUTH_URL = "https://oauth.platform.intuit.com/oauth2/v1/tokens/bearer"

In [17]:
def get_auth_header():
    """Create the Basic Auth header for token requests"""
    auth_string = f"{CLIENT_ID}:{CLIENT_SECRET}"
    encoded = base64.b64encode(auth_string.encode()).decode()
    return {
        "Authorization": f"Basic {encoded}",
        "Content-Type": "application/x-www-form-urlencoded"
    }

def refresh_access_token(refresh_token):
    """Get a new access token using the refresh token"""
    headers = get_auth_header()
    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token
    }
    
    response = requests.post(AUTH_URL, headers=headers, data=data)
    
    if response.status_code != 200:
        print(f"Error refreshing token: {response.status_code}")
        print(response.text)
        return None
    
    token_data = response.json()
    
    # Print token information
    print(f"Token Type: {token_data['token_type']}")
    print(f"Expires In: {token_data['expires_in']} seconds")
    
    # Store the new refresh token if provided
    if 'refresh_token' in token_data:
        print("New refresh token received - save this for future use!")
        print(f"New Refresh Token: {token_data['refresh_token']}")
    
    return token_data

In [18]:
def make_api_request(endpoint, access_token, method="GET", data=None):
    """Make a request to the QuickBooks API"""
    url = f"{BASE_URL}/v3/company/{COMPANY_ID}/{endpoint}"
    
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json",
        "Content-Type": "application/json"
    }
    
    if method == "GET":
        response = requests.get(url, headers=headers)
    elif method == "POST":
        response = requests.post(url, headers=headers, data=json.dumps(data))
    else:
        raise ValueError(f"Unsupported method: {method}")
    
    if response.status_code >= 400:
        print(f"Error: {response.status_code}")
        print(response.text)
    
    return response

def run_query(query, access_token):
    """Run a query against the QuickBooks API"""
    encoded_query = requests.utils.quote(query)
    endpoint = f"query?query={encoded_query}"
    return make_api_request(endpoint, access_token)

In [27]:
print("Getting access token...")
token_response = refresh_access_token(REFRESH_TOKEN)

Getting access token...
Token Type: bearer
Expires In: 3600 seconds
New refresh token received - save this for future use!
New Refresh Token: RT1-136-H0-1755970682gujh0lb5nvlfugovcotz


In [28]:
if token_response:
    access_token = token_response["access_token"]
    print("\nAccess token acquired successfully!")
    
    # Now let's try some API calls
    
    # 4.1 Get Company Info
    print("\n--- Company Info ---")
    company_info = make_api_request("companyinfo/" + COMPANY_ID, access_token)
    if company_info.status_code == 200:
        info = company_info.json()
        print(f"Company Name: {info['CompanyInfo']['CompanyName']}")
        print(f"Legal Name: {info['CompanyInfo'].get('LegalName', 'N/A')}")
        print(f"Address: {info['CompanyInfo'].get('CompanyAddr', {}).get('Line1', 'N/A')}")
    
    # 4.2 Get Vendors
    print("\n--- Vendors ---")
    query = "SELECT Id, DisplayName, Active FROM Vendor MAXRESULTS 10"
    vendors_response = run_query(query, access_token)
    
    if vendors_response.status_code == 200:
        vendors_data = vendors_response.json()
        if "QueryResponse" in vendors_data and "Vendor" in vendors_data["QueryResponse"]:
            vendors = vendors_data["QueryResponse"]["Vendor"]
            vendors_df = pd.DataFrame(vendors)
            display(vendors_df)
            
            # Store the first vendor ID for later use
            if vendors:
                first_vendor_id = vendors[0]["Id"]
                print(f"\nFirst vendor ID: {first_vendor_id} (saved for later)")
        else:
            print("No vendors found")
    
    # 4.3 Get Chart of Accounts
    print("\n--- Chart of Accounts ---")
    query = "SELECT Id, Name, AccountType, AccountSubType FROM Account WHERE Active = true MAXRESULTS 10"
    accounts_response = run_query(query, access_token)
    
    if accounts_response.status_code == 200:
        accounts_data = accounts_response.json()
        if "QueryResponse" in accounts_data and "Account" in accounts_data["QueryResponse"]:
            accounts = accounts_data["QueryResponse"]["Account"]
            accounts_df = pd.DataFrame(accounts)
            display(accounts_df)
            
            # Find an expense account for later use
            expense_accounts = [acct for acct in accounts if acct["AccountType"] == "Expense"]
            if expense_accounts:
                expense_account_id = expense_accounts[0]["Id"]
                print(f"\nExpense account ID: {expense_account_id} (saved for later)")
        else:
            print("No accounts found")
    
    # 4.4 Create a Simple Bill (uncomment when ready to test)
    print("\n--- Creating a Test Bill ---")
    if 'first_vendor_id' in locals() and 'expense_account_id' in locals():
        bill_data = {
            "VendorRef": {
                "value": first_vendor_id
            },
            "TxnDate": datetime.now().strftime("%Y-%m-%d"),
            "Line": [
                {
                    "DetailType": "AccountBasedExpenseLineDetail",
                    "Amount": 42.00,
                    "Description": "Test expense from API",
                    "AccountBasedExpenseLineDetail": {
                        "AccountRef": {
                            "value": expense_account_id
                        }
                    }
                }
            ]
        }
        
        bill_response = make_api_request("bill", access_token, method="POST", data=bill_data)
        
        if bill_response.status_code == 200:
            bill_result = bill_response.json()
            print(f"Bill created successfully!")
            print(f"Bill ID: {bill_result['Bill']['Id']}")
            print(f"Bill Doc Number: {bill_result['Bill'].get('DocNumber', 'N/A')}")
    else:
        print("Skipping bill creation - need vendor and account IDs")

else:
    print("Failed to get access token. Check your credentials.")


Access token acquired successfully!

--- Company Info ---
Company Name: Sandbox Company_US_2
Legal Name: Sandbox Company_US_2
Address: 123 Sierra Way

--- Vendors ---


,sparse,Id,DisplayName,Active
0,True,56,Bob's Burger Joint,True
1,True,30,Books by Bessie,True
2,True,31,Brosnahan Insurance Agency,True
3,True,32,Cal Telephone,True
4,True,33,Chin's Gas and Oil,True
5,True,34,Cigna Health Care,True
6,True,35,Computers by Jenni,True
7,True,36,Diego's Road Warrior Bodyshop,True
8,True,37,EDD,True
9,True,38,Ellis Equipment Rental,True



First vendor ID: 56 (saved for later)

--- Chart of Accounts ---


,Name,AccountType,AccountSubType,sparse,Id
0,Accounts Payable (A/P),Accounts Payable,AccountsPayable,True,33
1,Accounts Receivable (A/R),Accounts Receivable,AccountsReceivable,True,84
2,Advertising,Expense,AdvertisingPromotional,True,7
3,Arizona Dept. of Revenue Payable,Other Current Liability,GlobalTaxPayable,True,89
4,Automobile,Expense,Auto,True,55
5,Fuel,Expense,Auto,True,56
6,Bank Charges,Expense,BankCharges,True,8
7,Billable Expense Income,Income,ServiceFeeIncome,True,85
8,Board of Equalization Payable,Other Current Liability,GlobalTaxPayable,True,90
9,Checking,Bank,Checking,True,35



Expense account ID: 7 (saved for later)

--- Creating a Test Bill ---
Bill created successfully!
Bill ID: 145
Bill Doc Number: N/A


In [29]:
def explore_structure(json_data, prefix=""):
    """
    Recursively explore and print the structure of a JSON object
    """
    if isinstance(json_data, dict):
        for key, value in json_data.items():
            if isinstance(value, (dict, list)):
                print(f"{prefix}{key}:")
                explore_structure(value, prefix + "  ")
            else:
                print(f"{prefix}{key}: {type(value).__name__}")
    elif isinstance(json_data, list) and json_data:
        print(f"{prefix}[0]:")
        explore_structure(json_data[0], prefix + "  ")

# Example: Explore vendor structure
if 'vendors_data' in locals() and "QueryResponse" in vendors_data and "Vendor" in vendors_data["QueryResponse"]:
    print("\n--- Vendor Data Structure ---")
    explore_structure(vendors_data["QueryResponse"]["Vendor"][0])


--- Vendor Data Structure ---
sparse: bool
Id: str
DisplayName: str
Active: bool
